## ASSOCIATION RULES

"O ürünü alanlar bu ürünü de aldı" gibi cümlelere sık sık denk gelmekteyiz. İşte birliktelik analizi tam da bu kısımda karşımıza çıkmaktadır. 
Tanım olarak, veri tabanları içerisinde ürünlerin birlikteliğini inceleyip, bu birlikteliklere göre tavsiye veya stratejiler alma imkanı sağlayan analiz türüdür.

Birliktelik analizi uygulamaları e-ticaret, perakendecilik gibi sektörlerde kendine yer bulmuştur. 

Birliktelik analizinde çeşitli yöntemler kullanılmaktadır.En çok karşımıza çıkan "Apriori Algritması" dır.

Apriori Algoritması

Birliktelik kuralı analizi bazı metrikler incelenerek gerçekleştirilir:

* Support : Alışveriş yapanların sepetinde ürünler kaç defa birlikte geçmiş
Support(X, Y) = Freq (X,Y)/ N    

* Confidence  : X alanların yüzde şu kadarı Y ürününü de alacak.
Confidence(X, Y) = Freq (X,Y) / Freq(X)

* Lift : Bir sepette X ürünü olması durumunda Y ürününün satın alınmasının kaç kat arttığını gösterir.
Lift = Support (X, Y) / (Support(X) * Support(Y)) 

X: ürün Y: ürün N: toplam alışveriş



## DATA UNDERSTANDING 

* Import işlemleri

In [ ]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


* Datayı okuma

In [ ]:
df = pd.read_csv('../input/supermarket/GroceryStoreDataSet.csv' , names =["Products"], header=None)
df.head() #Veri setinin ilk 5 gözlemi

In [ ]:
#Veri setinin boyutları
df.shape 

In [ ]:
#Veri setinin boyut sayısı
df.ndim

## DATA PREPROCESSING

In [ ]:
#unique değişkenlerin bulunup items olarak atanması
items = df["Products"].unique()
items

In [ ]:
#df içerisinden products değişkeni seçilip burada products değişkeninde yer alan ürünler bir arada iken birbirlerinden ayrılarak listelenmiştir.
items = list(df['Products'].apply(lambda x : x.split(',')))
items

In [ ]:
#müşterinin sepetinde hangi ürünlerin olup olmadığı ONE-HOT ENCODING ile boolean yapılır

te = TransactionEncoder()
te_items = te.fit(items).transform(items)
items_df = pd.DataFrame(te_items,columns=te.columns_)
items_df

### ASSOCIATION RULES 

In [ ]:
#Esik değerini support'a göre belirlemek için support hesabı yapılır. Çıktıya göre 83 farklı kombinasyon ortaya çıktı. (11 tane ürünümüz vardı.) 
#Yorum örneği : Ekmek bütün alışverişlerin % 65'inde  bulunmaktadır.
freq_items  = apriori(items_df,min_support=0.01,use_colnames=True)
freq_items

In [ ]:
final_tableau = association_rules(freq_items,metric="confidence",min_threshold = 0.6)
final_tableau

### Support ve confidence a göre seçimler yapma

* Support'u %30 dan büyük ve confidence değeri %60 dan fazla olan değerleri seçme;

In [ ]:
final_tableau[(final_tableau['confidence']>0.6) & (final_tableau['support']>0.3)]

* Support değeri %30 dan fazla ve confidence değeri %60 dan fazla olan ürünler bulunmamaktadır. Support değerini %20 yaparsak;

In [ ]:
final_tableau[(final_tableau['confidence']>0.6) & (final_tableau['support']>=0.2)]

### Final tablosu üzerinden yorum yapılır.

In [ ]:
final_tableau

In [ ]:
final_tableau.sort_values(by="support",ascending=False).head(6)

## Yorumlar

Sugar ve Bread ikilisi için incelediğimizde;

Support
* Alışverişlerde tek olarak sütün alınma olasılığı %25 iken ekmeğin alınması %65'dir. Süt ve ekmek ikilisi tüm alışverişlerin %20 sinde beraber alındığı görülmektedir.

Confidence
* Süt satıldığında ekmek satılma olasılığı 0,80 dir. Süt alanların %80'i ekmek de almaktadır. Bu sebeple süt ve ekmek genel ihtiyaçlar olduğundan birbirinden uzağa konulup araya da satmak istediğimiz ürünleri koyarsak müşteri birinden diğerine geçerken bunları farkedip almak isteyebilir.

Lift
* Süt satıldığında ekmek satılma olasılığı 1.23 kat artmaktadır.
